In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../input/dennisdutchlangdataset/dataset.csv")

df["title"] = df.text.apply(lambda x: x.split("\n")[0].strip())
df["text"] = df.text.apply(lambda x: "\n".join(x.split("\n")[1:]).strip())

In [ ]:
!pip install git+https://github.com/LIAAD/yake -q

In [ ]:
import yake
from tqdm.notebook import tqdm

all_keywords = []
for text in tqdm(df.text.tolist()):

    kw_extractor = yake.KeywordExtractor(lan="nl")
    extracted = kw_extractor.extract_keywords(text)

    keywords = []

    for kw in extracted:
        keywords.append(kw)
        
    all_keywords.append(keywords)
    
df["keywords"] = all_keywords

In [ ]:
model_name = "yhavinga/gpt2-medium-dutch"

keyword_number = 3

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
df.columns

In [ ]:
import re

training_texts = []

for record in df.to_dict("record"):
    
    keywords = [i[0].strip(",") for i in record["keywords"][:keyword_number]]
    text = record["text"]
    text = re.sub("[\n]+", "\n\n", text)
    title = record["title"]
    
    training_text = title + "\n" + ", ".join(keywords) + "\n\n#######\n\n" + text
    training_text = tokenizer.decode(tokenizer.encode(training_text)[:1024])
    
    training_texts.append(training_text)

In [ ]:
df["training_text"] = training_texts

train = df.iloc[:295]
test = df.iloc[295:]

In [ ]:
text = "\n\n\n---\n\n\n".join(training_texts)

f = open("train.txt", "w", encoding="utf-8")
f.write(text)
f.close()

In [ ]:
!pip install simpletransformers -q

In [ ]:
import logging

from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



In [ ]:
from simpletransformers.language_modeling import LanguageModelingModel


train_args = {
    "reprocess_input_data": True,
    "num_train_epochs": 2,
    "save_eval_checkpoints": False,
    "block_size": 200,
    "max_seq_length": 200,
    # "save_model_every_epoch": False,
    "mlm": False,
    "learning_rate": 1e-4,
    "train_batch_size": 1,
    "gradient_accumulation_steps":2,
    "dataset_type": "simple",
    "logging_steps": 100,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 3000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": True,
    "sliding_window": True,
    "use_multiprocessing": False,
    "evaluate_during_training": False,
    "num_beams": 2
}

In [ ]:
#TODO: Find a way to resize tokenizer instead of using standard one
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


class SimpleDataset(Dataset):
    def __init__(
        self,
        tokenizer,
        file_path,
    ):

        with open(file_path, encoding="utf-8") as f:
            lines = [
                tokenizer.encode(
                    ele,
                )
                for ele in f.read().split("\n\n\n---\n\n\n")
            ]

        self.examples = lines


    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


    
def load_and_cache_examples(self, file_path, verbose=None, evaluate=None, silent=None):
    return SimpleDataset(self.tokenizer, file_path)


LanguageModelingModel.load_and_cache_examples = load_and_cache_examples

In [ ]:
model = LanguageModelingModel("gpt2", model_name, args=train_args)

In [ ]:
import torch

model.train_model(
    "train.txt", 
)

In [ ]:
import logging
from simpletransformers.language_generation import LanguageGenerationModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = LanguageGenerationModel("gpt2", "outputs", args={"max_length": 512, "top_k":50, "num_return_sequences":1})

In [ ]:
written_texts = test.training_text.apply(lambda x: x.split("#######")[0] + "#######").tolist()
texts = []

for written_text in written_texts:
    texts.append(model.generate(written_texts[0], verbose=False)[0])

In [ ]:
written_texts

In [ ]:
output_text = ""
for text in texts:
    
    splitted = text.split("#######")
    output_text += f"Input: {splitted[0]}\n----\noutput: {splitted[1]} \n\n\n ****************** \n\n\n"

In [ ]:
file = open("examples_01.txt", "w", encoding="utf-8")
file.write(output_text)
file.close()